In [1]:
import torch
torch.cuda.empty_cache()

# Import Libraries

In [2]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import matplotlib.pyplot as plt
import random
import copy
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score, accuracy_score
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
#from efficientnet_pytorch import model as enet

import cv2
import timm

from tqdm import tqdm
from tqdm.notebook import tqdm

import albumentations as A
from albumentations import (
    Compose, OneOf, Normalize, CenterCrop, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, RandomRotate90, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, HueSaturationValue, CoarseDropout
    )
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

import logging
import time
from contextlib import contextmanager

# Config

In [3]:
class CFG:
    DATA_PATH = '../input/plant-pathology-2021-fgvc8/'
    TEST_PATH = DATA_PATH + 'test_images/'
    TRAIN_PATH = DATA_PATH + 'train_images/'
    TRAIN_CSV_PATH = DATA_PATH + 'train.csv'
    SUB_CSV_PATH = DATA_PATH + 'sample_submission.csv'
    
    MODEL_ARCH ='vit_base_patch16_224'

    IMG_SIZE = 224
    EPOCH = 10
    BATCH_SIZE = 32
    DEBUG_SIZE = 0.1
    RANDOM_STATE = 1234
    CLASS_THRESHOLD = 0.4
    DROPOUT = .4
    LR = 5e-4
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    LABELS = ['healthy','complex','rust','frog_eye_leaf_spot','powdery_mildew','scab']
    N_CLASS = 12
    LABELS_DICT = {
    0: 'healthy', 
    1: 'scab',
    2: 'rust',
    3: 'frog_eye_leaf_spot',
    4: 'complex', 
    5: 'powdery_mildew',
    6: 'scab frog_eye_leaf_spot',
    7: 'scab frog_eye_leaf_spot complex',
    8: 'frog_eye_leaf_spot complex',
    9: 'rust frog_eye_leaf_spot',
    10: 'rust complex',
    11: 'powdery_mildew complex'
    }


In [4]:
train_df = pd.read_csv(CFG.TRAIN_CSV_PATH)
sub_df = pd.read_csv(CFG.SUB_CSV_PATH)
sub = pd.read_csv(CFG.SUB_CSV_PATH)

# Seed

In [5]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

@contextmanager
def timer(name, logger=None, level=logging.DEBUG):
    print_ = print if logger is None else lambda msg: logger.log(level, msg)
    t0 = time.time()
    print_(f'[{name}] start')
    yield
    print_(f'[{name}] done in {time.time() - t0:.0f} s')

In [6]:
def to_numpy(tensor):
    """Auxiliary function to convert tensors into numpy arrays
    """
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


# Dataset

In [7]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = sub_df
        self.image_names = sub_df['image'].values
        self.labels = sub_df['labels'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        label = self.labels[idx]
        image_path = CFG.TEST_PATH + image_name
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

# Transform

In [8]:
def get_transforms(data):
    
    if data == 'train':
        return Compose([
            A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
            A.RandomResizedCrop(CFG.IMG_SIZE, CFG.IMG_SIZE),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.RandomBrightnessContrast(p=0.5),

            A.Normalize(),
            ToTensorV2(),
        ])
    elif data == 'valid':
        return Compose([
            Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
            Normalize(),
            ToTensorV2(),
        ])

In [9]:
test_dataset = TestDataset(sub_df, transform = get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [10]:
test_dataset[0]

# Create model

In [11]:
class ViT(nn.Module):
    """
        Base class for all neural network modules.
        Your models should also subclass this class.
        Modules can also contain other Modules, allowing to nest them in a tree structure.
        You can assign the submodules as regular attributes:
    """
    def __init__(self):
        super(ViT, self).__init__()
        self.vit = timm.create_model(model_name = CFG.MODEL_ARCH,pretrained = False)
        
        #in_features = self.vit.fc.in_features
        #self.efficientnetb2.classifier = nn.Linear(in_features, len(CFG.LABELS))#(input_size,output_size)
        self.vit.head = nn.Linear(self.vit.head.in_features, CFG.N_CLASS)

#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
    def forward(self, x):
        x = self.vit(x)
        return x
    
model = ViT()

model.load_state_dict(torch.load("../input/myvitpt/my_vit_model.pt"))

# Prediction

In [12]:
#model.cuda()
model.eval()

predictions = []
for inputs in test_loader:
    inputs = inputs.to(CFG.DEVICE)
    with torch.no_grad():
        model = model.to(CFG.DEVICE)
        outputs = model(inputs)
        preds = outputs.argmax(1).detach().cpu().numpy()
        predictions.append(preds)
outputs,preds

# Submission

In [13]:
sub['labels'] = np.concatenate(predictions)
sub = sub.replace({"labels": CFG.LABELS_DICT})
sub.to_csv('submission.csv', index=False)
sub.head()